In [ ]:
Descripción

Configuración del ambiente
Para empezar, puedes utilizar un entorno virtual como Google Colaboratory, o si deseas, puedes usar el editor
Python de tu preferencia, sólo asegúrate de tener Python 3.x instalado en tu computadora.

En el proyecto usaremos 5 variables que serán manipuladas dentro de varias funciones, estas variables son 
conocidas como variables globales, es necesario que las definas aquí de la siguiente forma:
global 
df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision

También necesitarás instalar algunas librerías de Python que son esenciales para este proyecto, como Pandas, 
Numpy, Matplotlib, etc.

Para estar seguros de que tenemos todo listo en nuestro Ambiente, podemos ejecutar las siguientes líneas de código:

!python -V
print('------')
!pip show Pandas | grep 'Name\|Version'
print('------')
!pip show Numpy | grep 'Name\|Version'
print('------')
!pip show Matplotlib | grep 'Name\|Version'
El resultado debería ser las bibliotecas y sus versiones que tenemos instaladas en nuestro Ambiente, si tienes
algún error revisa tu configuración, talvez falte instalar alguna biblioteca.



# 1. Configuración del ambiente

In [9]:
!python -V
print('------')
!pip show Pandas | grep 'Name\|Version'
print('------')
!pip show Numpy | grep 'Name\|Version'
print('------')
!pip show Matplotlib | grep 'Name\|Version'

Python 3.7.6
------
Name: pandas
Version: 1.3.5
------
Name: numpy
Version: 1.18.1
------
Name: matplotlib
Version: 3.1.3


## 1.1 definir variables globales

In [75]:
# Definir variables globales
global df_bitcoin, precio_actual, tendencia, media_bitcoin, algoritmo_decision


# 2. Obtención de datos

In [ ]:
Descripción

Obtención de datos
Crea una funcion importar_base_bitcoin(), dentro de ella, define nuevamente las variables globales(ver Paso1) y
escribe tu código:

Utiliza la biblioteca yfinance de Python para extraer el histórico de precios del Bitcoin en dólares BTC-USD, 
deberás extraer el histórico de los últimos 7 días en intervalos de 5 minutos, este histórico deberás guardarlo en
el dataframe df_bitcoin.
Crea una funcion extraer_tendencias(), dentro de ella, define nuevamente las variables globales(ver Paso1) y 
escribe tu código:

Utilizando la biblioteca BeautifulSoup deberás realizar Web Scraping de la página https://coinmarketcap.com/ 
para extraer el precio actual del Bitcoin BTC en dólares USD y la variación de su precio en la última hora 1h % . 
El precio deberás convertirlo a float y guardarlo en la variable llamada precio_actual. Ahora, en la variable 
tendencia guarda el valor de 'baja'si la variación del precio es negativa, sino, guarda el valor de 'alta'.

Apoyo
Puedes encontrar más información de como instalar y usar la biblioteca yfinance en su página oficial: yfinance.
Además de los recursos mencionados en el card ‘Para saber más’ del Trello, puedes encontrar información 
complementar de Web Scraping en el siguiente enlace: Web Scraping con Python: Cómo Extraer Datos de Sitios Web.

## 2.1 Precios historico del bitcoin

In [ ]:
import yfinance as yf
import datetime
import time
import pandas as pd

def importar_base_bitcoin():
    symbol = 'BTC-USD'

    # fecha de inicio y fin para los últimos 7 días
    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=7)

    # Descarga el precio histórico
    data = yf.download(symbol, start=start_date, end=end_date)

    # Pausa durante 5 minutos antes de la próxima actualización
    #time.sleep(300)  # 300 segundos = 5 minutos

    # Muestra los datos
    print("Última actualización:", datetime.datetime.now())
    print(data['Adj Close'])

# Ejecuta la función para comenzar a importar el historial y realizar cálculos
importar_base_bitcoin()


In [88]:
import yfinance as yf
import datetime
import time
import pandas as pd

def importar_base_bitcoin()
    symbol = 'BTC-USD'

    # fecha de inicio y fin para los últimos 7 días
    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=7)

    # Descarga el precio histórico
    data = yf.download(symbol, start=start_date, end=end_date)

    # Pausa durante 5 minutos antes de la próxima actualización
    time.sleep(300)  # 300 segundos = 5 minutos

    # Muestra los datos
    #print("Última actualización:", datetime.datetime.now())
    #print(data['Adj Close'])
   
    

importar_base_bitcoin()

SyntaxError: invalid syntax (<ipython-input-88-c89f5ce4a52f>, line 6)

In [ ]:
def importar_base_bitcoin():
    global df_bitcoin, precio_actual, tendencia, media_bitcoin, algoritmo_decision

    symbol = 'BTC-USD'

    # fecha de inicio y fin para los últimos 7 días
    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=7)

    # Descarga el precio histórico del bitcoin en intervalos de 5 minutos
    data = yf.download(symbol, start=start_date, end=end_date, interval='5m')

    # Pausa durante 5 minutos antes de la próxima actualización
    time.sleep(300)  # 300 segundos = 5 minutos
    
    df_bitcoin = pd.DataFrame(data)
    df_bitcoin
    
importar_base_bitcoin()

In [ ]:
#########

In [ ]:

df

In [ ]:
global df_bitcoin, precio_actual, tendencia, media_bitcoin, algoritmo_decision

symbol = 'BTC-USD'

# fecha de inicio y fin para los últimos 7 días
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=7)

# Descarga el precio histórico en intervalos de 5 minutos
data = yf.download(symbol, start=start_date, end=end_date, interval='5m')

# Pausa durante 5 minutos antes de la próxima actualización
#time.sleep(300)  # 300 segundos = 5 minutos
    


In [ ]:
df_bitcoin = pd.DataFrame(data)
df_bitcoin
    

In [125]:
df_bitcoin.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1989.000000,1989.000000,1989.000000,1989.000000,1989.000000,1.989000e+03
mean,27647.861714,27656.465503,27638.713525,27647.375014,27647.375014,1.212721e+07
std,317.056982,315.984285,318.950236,317.989180,317.989180,2.726747e+07
min,26561.099609,26595.152344,26561.099609,26569.714844,26569.714844,0.000000e+00
25%,27476.507812,27488.490234,27464.070312,27472.183594,27472.183594,0.000000e+00
50%,27669.689453,27677.427734,27662.369141,27669.033203,27669.033203,8.243200e+05
75%,27923.980469,27930.087891,27918.388672,27923.593750,27923.593750,1.244979e+07
max,28227.326172,28252.537109,28130.154297,28252.537109,28252.537109,3.868252e+08


In [ ]:
df_bitcoin = pd.DataFrame(data)
df_bitcoin

## 2.2 Scraping Precio bitcoin de coinmarketcap

In [65]:
from bs4 import BeautifulSoup
import requests


In [118]:
# HTTP Request
# store website in variable
#ua = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.62"}

ua = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.62 (compatible; Linux; Ubuntu; MacOS; Windows)"}
website = 'https://coinmarketcap.com/'

In [119]:
#Get Reques
response = requests.get(website)

In [120]:
#Status Code
response.status_code

200

In [121]:
#Soup Object
soup = BeautifulSoup(response.content, 'html.parser')

In [122]:

# Extraer el nombre de Bitcoin
nombre = soup.find('p', class_='sc-4984dd93-0 kKpPOn').text

# Extraer el precio de Bitcoin
precio = soup.find('div', class_='sc-a0353bbc-0 gDrtaY').text
#precio = soup.find('span').text

# Imprimir el nombre y el precio
print(f"Nombre: {nombre}")
print(f"Precio: {precio}")

Nombre: Bitcoin
Precio: $26,761.68


# 3. Limpieza de datos

In [ ]:
Descripción

Limpieza de datos
Crea una funcion limpieza_datos(), dentro de ella, define nuevamente las variables globales(ver Paso1) y escribe 
tu código:

Antes de limpiar la base, crea una copia de la base df_bitcoin para que realices la limpieza en esta nueva 
base(df_bitcoin_limpio), sin modificar la base original, además es importante entender su contenido:

Datetime: Es el índice del dataframe, y contiene la fecha y hora del intervalo seleccionado, no puede estar duplicado.

Open: Contiene el precio del Bitcoin al inicio del intervalo.

High: Contiene el precio más alto del Bitcoin durante el intervalo.

Low: Contiene el precio más bajo del Bitcoin durante el intervalo.

Close: Contiene el precio del Bitcoin al final del intervalo.

Volume: Contiene el total de transacciones hechas durante el intervalo.

Para la limpieza utilizaremos los atributos Datetime, Close y Volume.

Analiza la base df_bitcoin_limpio, identifica duplicados en el índice y trátalos para quedarte con sólo índices únicos.
Luego busca por valores nulos en la columna Close y trátalos.
Verifica que todos los registros de la base tengan un Volume de transacción mayor a 0, caso contrário, elimínalos.
Deberás identificar y eliminar los outliers en el precio del Bitcoin, columna Close, utiliza un gráfico de 
boxplot para identificarlos.
Filtra o selecciona únicamente los registros cuyo precio(Close) se encuentren entre el 1er cuartil(Q1) y el 3er 
cuartil(Q3) del boxplot.
Finalmente, calcula el precio promedio(Close) de esta selección y guárdala en la variable media_bitcoin.

# 4. Tomar decisiones

In [ ]:
Descripción

Tomar decisiones
Llegó el momento de construir tu algoritmo de decisión:

Crea una funcion tomar_decisiones(), dentro de ella, define nuevamente las variables globales(ver Paso1) y 
escribe tu código:

Rellena la variable algoritmo_decision con el valor resultante del siguiente criterio de decisión:
Si el precio actual es mayor/igual que el precio promedio y la tendencia es de baja, entonces guarda el valor ‘Vender’.
Si el precio actual es menor que el precio promedio y la tendencia es de alta, entonces guarda el valor ‘Comprar’.
Si ninguna de las 2 condiciones anteriores se cumple, entonces guarda el valor ''.
Extra
Este es un algoritmo de decisión simple que ayuda a los clientes inexpertos a conocer el mejor momento de comprar
o vender Bitcoin, ¿crees que existan otros indicadores en la base o en el site de noticias o algún criterio de 
decisión adicional que ayudaría a reforzar el algoritmo de decisión? Te invito a descubrir otras alternativas.



# 5. Visualización

In [ ]:
Descripción

Visualización
Crea una funcion visualizacion(), dentro de ella, define nuevamente las variables globales(ver Paso1) y escribe tu código:

Adiciona una nueva columna (Promedio) al dataframe original df_bitcoin y almacena el valor de nuestra variable
media_bitcoin.
Configura el tamaño del gráfico en una proporción de 16x5.
Adiciona un título al gráfico.
Usando el método plot()dibuja una línea en el gráfico, con los datos del índice y la columna Close de la base 
df_bitcoin.
Usando el método plot()dibuja una línea en el gráfico, con los datos del índice y la columna Promedio de la base
df_bitcoin.
Usando el método annotate()muestra un mensaje dentro del gráfico con la decisión calculada del algoritmo.
Finalmente, usando el método show()muestra en pantalla el gráfico que acabamos de configurar.
Apoyo
Puedes encontrar más información sobre la biblioteca matplotlib en su página oficial: matplotlib.pyplot

# 6. Automatización

In [ ]:
Descripción

Automatización
El trabajo de un Data Scientist sólo termina cuando la solución al problema está automatizada, esto permite 
soluciones automáticas para nuevos lotes de información.

Importa el método 'clear_output()', este método nos permite borrar los resultados de la pantalla antes de 
imprimir un nuevo gráfico, y así evitar tener más de un gráfico en la pantalla, impórtalo de la siguiente 
forma: from IPython.display import clear_output.
    
Importa también la biblioteca ‘time’ de la siguiente forma: import time, utilizaremos su método time.sleep(300) 
para interrumpir la ejecución del código cada 300 segundos o 5 minutos.

Finalmente construiremos un loop infinito para ejecutar permanentemente y en intervalos de 5minutos, las funciones 
que hemos construído en los pasos 2 al 5 de nuestro proyecto, de la siguiente forma:
    
    
while(True):
  clear_output()
  importar_base_bitcoin()
  extraer_tendencias()
  limpieza_datos()
  tomar_decisiones()
  visualizacion()
  time.sleep(300)

El resultado de este código será un gráfico mostrando el histórico de precios del Bitcoin y la decisión del 
algoritmo en tiempo real, basado en datos normalizados y limpios, que se actualizarán cada 5 minutos.

Conclusión
Este desafío sólo fue una demonstración de un proyecto real, donde vemos el poder de los datos para la toma de 
decisiones y la importancia que tiene el tratamiento de los datos para evitar tomar decisiones equivocadas, 
durante este Bootcamp aprenderás nuevas herramientas y habilidades que te llevarán a ti y a este proyecto a un 
nuevo nivel.

¡Ahora, tu misión, si decides aceptarla, es, superar este Robot! ¡Suerte!